In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df.head()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.2, random_state=42)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
import math

In [ ]:
class silver_bhaediya:

  def __init__(self,objective_function,n_iteration=50,population_size=50):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.objective_function=objective_function

  def initialize_population(self,X):
        self.individuals =  np.random.randint(0,2,size=(self.population_size,X.shape[1]))


  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.individuals):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.best_score:
              self.best_score=score
              self.best_score_dimension=individual

            scores.append(score)
        self.fitness_scores = scores


  def sigmoid(self,x):
    #return abs((2/math.pi)*np.arctan((math.pi/2)*x ))
    return 1/(1+np.exp( -1*(x) ))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid,method='linear'):
    self.initialize_population(X_train)
    self.best_score=np.inf
    self.best_score_dimension=np.ones(X_train.shape[1]) 
    delta_x=np.zeros((self.population_size,X_train.shape[1]))
    kbest=3



    for i in range(self.n_iteration):
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)

      if method=='linear':
        s=0.2-(0.2*((i+1)/self.n_iteration))
        e=0.1-(0.1*((i+1)/self.n_iteration))
        a=0.0+(0.2*((i+1)/self.n_iteration))
        c=0.0+(0.2*((i+1)/self.n_iteration))
        f=0.0+(2*((i+1)/self.n_iteration))
        w=0.9-(i+1)*(0.5)/(self.n_iteration)
      
      if method=='random':
        if 2*(i+1)<=self.n_iteration:
          pct=0.1-(0.2*(i+1)/self.n_iteration)
        else:
          pct=0
        w=0.9-(i+1)*(0.5)/(self.n_iteration)
        s=2*np.random.random()*pct
        a=2*np.random.random()*pct
        c=2*np.random.random()*pct
        f=2*np.random.random()
        e=pct
      
      if method=='quadraic':
        w=0.9-(i+1)*(0.5)/(self.n_iteration)
        s=0.2-(0.2*((i+1)/self.n_iteration))**2
        e=0.1-(0.1*((i+1)/self.n_iteration))**2
        a=0.0+(0.2*((i+1)/self.n_iteration))**2
        c=0.0+(0.2*((i+1)/self.n_iteration))**2
        f=0.0+(2*(i+1)/self.n_iteration)**2





      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)
      temp=individuals=self.individuals
      temp_2=(( (temp.reshape(temp.shape[0],1,temp.shape[1])-temp.reshape(1,temp.shape[0],temp.shape[1])).reshape(temp.shape[0]**2,temp.shape[1])**2) )
      temp_3=temp_2.reshape(temp.shape[0],temp.shape[0],temp.shape[1]).sum(axis=2)
      zz=np.argsort(temp_3 ) 
      cc=[ list(iter1[iter1!=iter2]) for iter1,iter2 in zip(zz,np.arange(temp.shape[0])) ]

      Si=-(np.repeat(individuals,kbest,axis=0).reshape(individuals.shape[0],kbest,individuals.shape[1])-individuals[np.array(cc)[:,:kbest]]).sum(axis=1)
      Ai=delta_x[np.array(cc)[:,:kbest]].sum(axis=1)/kbest
      Ci=individuals[np.array(cc)[:,:kbest]].sum(axis=1)/kbest-individuals
      Fi=self.best_score_dimension-self.individuals
      Ei=self.individuals+self.individuals[np.array(self.fitness_scores).argmax()]

      delta_x=s*Si+a*Ai+c*Ci+f*Fi+e*Ei+w*delta_x
      T=abs(delta_x)/np.sqrt(1+delta_x**2)

      self.individuals=np.where(np.random.random((self.population_size,X_train.shape[1]))<T,np.logical_not(self.individuals).astype(int),individuals)
    
      print('iteration ',i,' ',np.array(self.fitness_scores).min(),' |mean ',np.array(self.fitness_scores).mean(),' |best_score ',self.best_score)

    return self.best_score_dimension

    


In [ ]:
import scipy
from sklearn.metrics import log_loss

def score_model(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    # return log_loss(y_valid,model.predict_proba(X_valid))
    P = (model.predict(X_valid) != y_valid).mean()
    alpha=0.01
    # Compute for the objective function
    j = alpha*(X_valid.shape[1]/30)+(1-alpha)*P
    return j
gaf=silver_bhaediya(score_model,50,20)
res=gaf.fit(clf,X_train, y_train,X_test, y_test,'linear')

iteration  0   0.03773684210526316  |mean  0.0668421052631579  |best_score  0.03773684210526316
iteration  1   0.03840350877192982  |mean  0.06193070175438596  |best_score  0.03773684210526316
iteration  2   0.04007017543859649  |mean  0.06176228070175438  |best_score  0.03773684210526316
iteration  3   0.04007017543859649  |mean  0.06403333333333334  |best_score  0.03773684210526316
iteration  4   0.03873684210526315  |mean  0.05718684210526316  |best_score  0.03773684210526316
iteration  5   0.0293859649122807  |mean  0.06187982456140352  |best_score  0.0293859649122807
iteration  6   0.04007017543859649  |mean  0.06553684210526314  |best_score  0.0293859649122807
iteration  7   0.04007017543859649  |mean  0.07099824561403509  |best_score  0.0293859649122807
iteration  8   0.04007017543859649  |mean  0.06172982456140351  |best_score  0.0293859649122807
iteration  9   0.03071929824561403  |mean  0.06445263157894736  |best_score  0.0293859649122807
iteration  10   0.03840350877192982  